In [14]:

import numpy as np
import torch
from torchvision.models import resnet50,  resnet18
from torch import nn
from torch.utils.data import DataLoader

from sklearn.metrics.pairwise import cosine_similarity

In [15]:
from scipy.spatial.distance import cdist
import numpy as np

# Example vectors (rows in a matrix)
vectors = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# Compute pairwise Euclidean distances
distances = cdist(vectors, vectors, 'euclidean')

print(distances)


[[ 0.          5.19615242 10.39230485]
 [ 5.19615242  0.          5.19615242]
 [10.39230485  5.19615242  0.        ]]


In [16]:
test_dataset = torch.load("data/test_dataset.pt")

In [17]:
import os

baseline = resnet50(pretrained=True)
    
baseline.fc = nn.Linear(2048, 910)
    
if os.path.exists(f"models/baseline.pt"):
    baseline.load_state_dict(torch.load(f"models/baseline.pt"))

baseline.fc = torch.nn.Identity()

c:\Users\sebas\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sebas\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
from torchvision import models

triplet = models.alexnet(pretrained=True)
    
triplet.fc = torch.nn.Identity()
    
if os.path.exists(f"models/alexnet18.pt"):
    triplet.load_state_dict(torch.load("models/alexnet18.pt"))


c:\Users\sebas\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sebas\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
def get_accuracy(embeddings, targets):
    embeddings = np.array(embeddings)
    targets = np.array(targets)
    
    # Get cosine similarity for all embeddings
    similarity_matrix = cosine_similarity(embeddings)
    
    # Fill diagonal with inf
    np.fill_diagonal(similarity_matrix, -np.inf)
    
    # Find the index of max sim for each embedding
    most_similar_indices = np.argmax(similarity_matrix, axis=1)
    
    # Extract the labels of the most similar items
    predicted_labels = [targets[i] for i in most_similar_indices]
    n_correct = sum(1 for true, pred in zip(targets, predicted_labels) if true == pred)
    
    accuracy = 100 * (n_correct / len(embeddings))
    
    return accuracy, predicted_labels

In [20]:
def get_embeddings(model, dataloader, device):
    embeddings = []
    targets = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            data = inputs.to(device)
            label = labels.to(device)
            batch_emb, batch_lab = [], []            
            
            emb = model(data)
            
            batch_emb.extend(emb.cpu())
            batch_lab.extend(label.cpu().tolist())
            
            embeddings.extend(batch_emb)
            targets.extend(batch_lab)
            
    return embeddings, targets

In [10]:
test_dataloader = DataLoader(test_dataset, 128, True)

In [11]:
device = "cuda"
baseline = baseline.to(device)

baseline_emb, baseline_targ = get_embeddings(baseline, test_dataloader, "cuda")

KeyboardInterrupt: 

In [ ]:
for i in range(len(baseline_emb)):
    baseline_emb[i] = baseline_emb[i].detach().numpy()

In [ ]:
type(baseline_emb[0])

numpy.ndarray

In [ ]:
# baseline_accuracy, baseline_preds = get_accuracy(baseline_emb, baseline_targ)

In [ ]:
# baseline_accuracy

80.91722239864507

In [24]:
triplet = triplet.to(device)
triplet_emb, triplet_targ = get_embeddings(triplet, test_dataloader, "cuda")

for i in range(len(triplet_emb)):
    triplet_emb[i] = triplet_emb[i].detach().numpy()

triplet_accuracy, triplet_preds = get_accuracy(triplet_emb, triplet_targ)

In [26]:
print(f"Test set with {len(test_dataset)} samples of {len(set(test_dataset.labels))} classes.")
# print(f"Baseline model openset performance: {baseline_accuracy}")
print(f"Triplet model openset performance: {triplet_accuracy}")

Test set with 75576 samples of 400 classes.
Triplet model openset performance: 11.629353233830846


TODO: Compare best model performances to majority classifier and specify that the bad model still has some knowledge. Also use random classifier